In [ ]:
import torch
import torch.nn as nn
import warnings
warnings.filterwarnings("ignore")
import torch.optim as optim
from torchvision import transforms, datasets, models

transform= transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])

])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)


['Corn_(maize)___Common_rust_',
 'Corn_(maize)___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Target_Spot',
 'Tomato___healthy']

In [ ]:
model=nn.Sequential(
    models.resnet18(pretrained=True),
    nn.Linear(1000, 512),
    nn.ReLU(),
    nn.Linear(512, 38),
    nn.LogSoftmax(dim=1)
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 3


In [ ]:
# for epoch in range(num_epochs):
#     running_loss = 0.0
#     for images, labels in train_loader:
#         images, labels = images.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#     torch.save(model.state_dict(), f'plant_disease_model_epoch{epoch}.pth')
#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [1/3], Loss: 0.4357
Epoch [2/3], Loss: 0.2056
Epoch [3/3], Loss: 0.1424


In [18]:

from PIL import Image

import torch
from torchvision import transforms
from torchvision import models
import torch.nn as nn

transform= transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])

])


model_loaded = nn.Sequential(
    models.resnet18(pretrained=True),
    nn.Linear(1000, 512), 
    nn.ReLU(),
    nn.Linear(512, 38),
    nn.LogSoftmax(dim=1)
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Move model to the correct device
model_loaded.to(device)
# Load the saved model weights
model_loaded.load_state_dict(torch.load('plant_disease_model_epoch2.pth', map_location=device))
model_loaded.eval()

train_dataset = datasets.ImageFolder('data/train', transform=transform)
import pickle
with open('train_dataset_classes.pkl', 'wb') as f:
    pickle.dump(train_dataset.classes, f)


# Load and preprocess the image
img_path = 'data/train/Tomato___Target_Spot/0a458dfc-b513-44f2-a3ce-dab4c3adb939___Com.G_TgS_FL 8166_new30degFlipLR.JPG'
img = Image.open(img_path).convert('RGB')
img_tensor = transform(img).unsqueeze(0).to(device)

# Load model weights
# model.load_state_dict(torch.load('plant_disease_model_epoch2.pth'))
# model.eval()

# Predict
classes = train_dataset.classes
with torch.no_grad():
    output = model_loaded(img_tensor)
    pred = output.argmax(dim=1)
print(f"Predicted class: {pred.item()}")
print(f"Predicted class name: {classes[pred.item()]}")

Predicted class: 10
Predicted class name: Tomato___Target_Spot
